In [1]:
import pandas as pd
import os
from rapidfuzz import process as rprocess, fuzz
from tqdm import tqdm

In [2]:
os.chdir("..")
print(os.getcwd())

c:\Users\valen\Desktop\etl_workshop002


In [3]:
spotify_df = pd.read_csv("data/clean/spotify_artists.csv")
grammy_df = pd.read_csv("data/clean/artist_grammy_nomination.csv")
lastfm_df = pd.read_csv("data/clean/lastfm_artists.csv")


In [4]:
tqdm.pandas()

df_spotify = spotify_df.copy()
df_grammy = grammy_df.copy()
df_lastfm = lastfm_df.copy()

def normalize_artist_name(name):
    if pd.isna(name):
        return ""
    return str(name).strip().lower()

df_spotify['artist'] = df_spotify['artist'].apply(normalize_artist_name)
df_grammy['artist'] = df_grammy['artist'].apply(normalize_artist_name)
df_lastfm['artist'] = df_lastfm['artist'].apply(normalize_artist_name)

spotify_artists = df_spotify['artist'].unique().tolist()

def get_best_match(name, choices, threshold=90):
    result = rprocess.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= threshold:
        return result[0]
    return None

df_grammy['matched_artist'] = df_grammy['artist'].progress_apply(lambda x: get_best_match(x, spotify_artists))
df_lastfm['matched_artist'] = df_lastfm['artist'].progress_apply(lambda x: get_best_match(x, spotify_artists))

df_grammy_reduced = df_grammy[['matched_artist', 'nominations', 'year_with_most_nominations',
                               'most_common_category', 'most_common_award_class']]
df_lastfm_reduced = df_lastfm[['matched_artist', 'lastfm_listeners', 'lastfm_playcount',
                               'similar_1', 'similar_2', 'similar_3']]


100%|██████████| 17632/17632 [02:08<00:00, 137.24it/s]


In [5]:
df_merge1 = df_spotify.merge(df_grammy_reduced, how='left', left_on='artist', right_on='matched_artist')

df_final = df_merge1.merge(df_lastfm_reduced, how='left', left_on='artist', right_on='matched_artist', suffixes=('', '_lastfm'))

df_final['nominations'] = df_final['nominations'].fillna(0).astype(int)

df_final.drop(columns=['matched_artist', 'matched_artist_lastfm'], errors='ignore', inplace=True)

In [6]:
print("Shape final del dataframe:", df_final.shape)
df_final.head()

Shape final del dataframe: (17677, 20)


,artist,total_tracks,spotify_popularity,danceability,energy,speechiness,instrumentalness,pct_explicit_tracks,duration_min,feature_pct,spotify_track_genre,nominations,year_with_most_nominations,most_common_category,most_common_award_class,lastfm_listeners,lastfm_playcount,similar_1,similar_2,similar_3
0,!nvite,2,23.0,High,Low,Low,Vocal,0.0,2.285,0.0,study,0,NaN,NaN,NaN,8030.0,19556.0,Bayleaf Manilla,Smiloh,Tentoe
1,#kids,1,34.0,Medium,Medium,Low,Instrumental,0.0,1.730,100.0,children,0,NaN,NaN,NaN,815.0,4431.0,Lullabies In Nature,Modern Children's Songs,Zouzounia TV
2,$affie,2,41.0,High,Low,Low,Instrumental,0.0,2.195,50.0,study,0,NaN,NaN,NaN,12512.0,32051.0,Ricki Wu,Lobby Lingu!n!,Exxecs
3,&me,1,50.0,High,Medium,Low,Instrumental,0.0,6.420,100.0,deep-house,0,NaN,NaN,NaN,138173.0,1164546.0,Rampa,Adriatique,Adam Port
4,'falsettos' 2016 broadway company,1,30.0,Low,Low,Low,Vocal,0.0,5.140,0.0,show-tunes,0,NaN,NaN,NaN,21687.0,487458.0,Anthony Rosenthal,Stephanie J. Block,Christian Borle


In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17677 entries, 0 to 17676
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   artist                      17677 non-null  object 
 1   total_tracks                17677 non-null  int64  
 2   spotify_popularity          17677 non-null  float64
 3   danceability                17677 non-null  object 
 4   energy                      17677 non-null  object 
 5   speechiness                 17677 non-null  object 
 6   instrumentalness            17677 non-null  object 
 7   pct_explicit_tracks         17677 non-null  float64
 8   duration_min                17677 non-null  float64
 9   feature_pct                 17677 non-null  float64
 10  spotify_track_genre         17677 non-null  object 
 11  nominations                 17677 non-null  int64  
 12  year_with_most_nominations  657 non-null    float64
 13  most_common_category        657

In [8]:
df_final["artist"].unique()

array(['!nvite', '#kids', '$affie', ..., '黃小琥', '黃敏華', '龍藏ryuzo'],
      shape=(17638,), dtype=object)

In [9]:
print(f"Number of duplicates: {df_final.duplicated().sum()}")

Number of duplicates: 18


In [10]:
df_final = df_final.drop_duplicates()

In [11]:
print(f"Number of duplicates: {df_final.duplicated(subset='artist').sum()}")

Number of duplicates: 21


In [12]:
duplicates = df_final[df_final.duplicated(subset='artist', keep=False)]
duplicates.head()

,artist,total_tracks,spotify_popularity,danceability,energy,speechiness,instrumentalness,pct_explicit_tracks,duration_min,feature_pct,spotify_track_genre,nominations,year_with_most_nominations,most_common_category,most_common_award_class,lastfm_listeners,lastfm_playcount,similar_1,similar_2,similar_3
590,ali farka touré,12,11.25,Medium,Low,Low,Vocal,0.0,5.534167,66.666667,afrobeat,1,1994.0,Best World Music Album,album,266779.0,3969159.0,Ali Farka Touré & Toumani Diabaté,Ali Farka Touré and Ry Cooder,Vieux Farka Touré
591,ali farka touré,12,11.25,Medium,Low,Low,Vocal,0.0,5.534167,66.666667,afrobeat,2,2005.0,Best Traditional World Music Album,album,266779.0,3969159.0,Ali Farka Touré & Toumani Diabaté,Ali Farka Touré and Ry Cooder,Vieux Farka Touré
1421,baby sleep white noise,1,36.00,Low,Low,Low,Instrumental,0.0,1.310000,0.000000,sleep,0,NaN,NaN,NaN,524.0,78942.0,WHITE NOISE FOR SLEEP,Baby Sleep Music,Pink Noise Sleep
1422,baby sleep white noise,1,36.00,Low,Low,Low,Instrumental,0.0,1.310000,0.000000,sleep,0,NaN,NaN,NaN,30111.0,9552210.0,Erik Eriksson,White Noise For Babies,littleONES
1716,berliner philharmoniker,4,0.25,Low,Low,Low,Instrumental,0.0,3.330000,100.000000,german,2,1999.0,Best Classical Vocal Performance,track,155858.0,2002117.0,Wiener Philharmoniker,Chicago Symphony Orchestra,Sir Simon Rattle


In [13]:
df_final = df_final.sort_values('nominations', ascending=False)
df_final = df_final.drop_duplicates(subset='artist', keep='first')

In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17638 entries, 2778 to 5
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   artist                      17638 non-null  object 
 1   total_tracks                17638 non-null  int64  
 2   spotify_popularity          17638 non-null  float64
 3   danceability                17638 non-null  object 
 4   energy                      17638 non-null  object 
 5   speechiness                 17638 non-null  object 
 6   instrumentalness            17638 non-null  object 
 7   pct_explicit_tracks         17638 non-null  float64
 8   duration_min                17638 non-null  float64
 9   feature_pct                 17638 non-null  float64
 10  spotify_track_genre         17638 non-null  object 
 11  nominations                 17638 non-null  int64  
 12  year_with_most_nominations  646 non-null    float64
 13  most_common_category        646 non-n

In [15]:
both_null = df_final.loc[df_final["lastfm_listeners"].isna() & df_final["lastfm_playcount"].isna()]
both_null

,artist,total_tracks,spotify_popularity,danceability,energy,speechiness,instrumentalness,pct_explicit_tracks,duration_min,feature_pct,spotify_track_genre,nominations,year_with_most_nominations,most_common_category,most_common_award_class,lastfm_listeners,lastfm_playcount,similar_1,similar_2,similar_3
2911,chrystian & ralf,5,46.800000,Medium,Medium,Low,Vocal,0.0,3.726000,0.000000,sertanejo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2966,clarence white,4,23.000000,Medium,Medium,Low,Instrumental,0.0,1.342500,0.000000,bluegrass,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985,clay walker,2,0.000000,Medium,Medium,Low,Vocal,0.0,2.720000,0.000000,country,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4189,dombresky,4,43.500000,Medium,High,Low,Vocal,0.0,3.402500,75.000000,deep-house,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9554,maikel delacalle,2,0.000000,High,High,Low,Vocal,0.0,3.660000,50.000000,latino,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9956,maunalua,2,24.000000,Medium,Low,Low,Vocal,0.0,4.915000,0.000000,guitar,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9991,maxel,1,30.000000,High,High,Low,Vocal,0.0,5.110000,0.000000,disco,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9992,maxence cyrin,5,38.600000,Low,Low,Low,Instrumental,0.0,2.690000,0.000000,ambient,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9993,maxi rozh,1,27.000000,High,High,Low,Vocal,0.0,2.720000,0.000000,groove,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9994,maxida märak,3,39.000000,High,Medium,Low,Vocal,0.0,2.856667,33.333333,swedish,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_final = df_final[~(df_final["lastfm_listeners"].isna() & df_final["lastfm_playcount"].isna())]

In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17622 entries, 2778 to 5
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   artist                      17622 non-null  object 
 1   total_tracks                17622 non-null  int64  
 2   spotify_popularity          17622 non-null  float64
 3   danceability                17622 non-null  object 
 4   energy                      17622 non-null  object 
 5   speechiness                 17622 non-null  object 
 6   instrumentalness            17622 non-null  object 
 7   pct_explicit_tracks         17622 non-null  float64
 8   duration_min                17622 non-null  float64
 9   feature_pct                 17622 non-null  float64
 10  spotify_track_genre         17622 non-null  object 
 11  nominations                 17622 non-null  int64  
 12  year_with_most_nominations  646 non-null    float64
 13  most_common_category        646 non-n

In [18]:
df_final.to_csv("data/clean/merged_data.csv", index=False)